# Projeto 2 - Classificador Automático de Sentimento

Você foi contratado por uma empresa parar analisar como os clientes estão reagindo a um determinado produto no Twitter. A empresa deseja que você crie um programa que irá analisar as mensagens disponíveis e classificará como "relevante" ou "irrelevante". Com isso ela deseja que mensagens negativas, que denigrem o nome do produto, ou que mereçam destaque, disparem um foco de atenção da área de marketing.<br /><br />
Como aluno de Ciência dos Dados, você lembrou do Teorema de Bayes, mais especificamente do Classificador Naive-Bayes, que é largamente utilizado em filtros anti-spam de e-mails. O classificador permite calcular qual a probabilidade de uma mensagem ser relevante dadas as palavras em seu conteúdo.<br /><br />
Para realizar o MVP (*minimum viable product*) do projeto, você precisa implementar uma versão do classificador que "aprende" o que é relevante com uma base de treinamento e compara a performance dos resultados com uma base de testes.<br /><br />
Após validado, o seu protótipo poderá também capturar e classificar automaticamente as mensagens da plataforma.

## Informações do Projeto

Prazo: 13/Set até às 23:59.<br />
Grupo: 1 ou 2 pessoas.<br /><br />
Entregáveis via GitHub: 
* Arquivo notebook com o código do classificador, seguindo as orientações abaixo.
* Arquivo Excel com as bases de treinamento e teste totalmente classificado.

**NÃO disponibilizar o arquivo com os *access keys/tokens* do Twitter.**


### Check 3: 

Até o dia 06 de Setembro às 23:59, o notebook e o xlsx devem estar no Github com as seguintes evidências: 
    * Conta no twitter criada.
    * Produto escolhido.
    * Arquivo Excel contendo a base de treinamento e teste já classificado.

Sugestão de leitura:<br />
http://docs.tweepy.org/en/v3.5.0/index.html<br />
https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/

___

## Preparando o ambiente

Instalando a biblioteca *tweepy* para realizar a conexão com o Twitter:

In [1]:
###
#%%capture
#
##Instalando o tweepy
#!pip install tweepy
###

Importando as Bibliotecas que serão utilizadas. Esteja livre para adicionar outras.

In [2]:
import tweepy
import math
import os.path
import pandas as pd
import json
import re
from random import shuffle

___
## Autenticando no  Twitter

Para realizar a captura dos dados é necessário ter uma conta cadastrada no twitter:

* Conta: @PJ2_CD17


1. Caso ainda não tenha uma: https://twitter.com/signup
1. Depois é necessário registrar um app para usar a biblioteca: https://apps.twitter.com/
1. Dentro do registro do App, na aba Keys and Access Tokens, anotar os seguintes campos:
    1. Consumer Key (API Key)
    1. Consumer Secret (API Secret)
1. Mais abaixo, gere um Token e anote também:
    1. Access Token
    1. Access Token Secret
    
1. Preencha os valores no arquivo "auth.pass"

**ATENÇÃO**: Nunca divulgue os dados desse arquivo online (GitHub, etc). Ele contém as chaves necessárias para realizar as operações no twitter de forma automática e portanto é equivalente a ser "hackeado". De posse desses dados, pessoas mal intencionadas podem fazer todas as operações manuais (tweetar, seguir, bloquear/desbloquear, listar os seguidores, etc). Para efeito do projeto, esse arquivo não precisa ser entregue!!!

In [3]:
#Dados de autenticação do twitter:

#Coloque aqui o identificador da conta no twitter: @PJ2_CD17

#leitura do arquivo no formato JSON
with open('auth.pass') as fp:    
    data = json.load(fp)

#Configurando a biblioteca. Não modificar
auth = tweepy.OAuthHandler(data['consumer_key'], data['consumer_secret'])
auth.set_access_token(data['access_token'], data['access_token_secret'])

___
## Coletando Dados

Agora vamos coletar os dados. Tenha em mente que dependendo do produto escolhido, não haverá uma quantidade significativa de mensagens, ou ainda poder haver muitos retweets.<br /><br /> 
Configurando:

In [4]:
#Produto escolhido:
produto = 'Drawing Tablet'

#Quantidade mínima de mensagens capturadas:
n = 500
#Quantidade mínima de mensagens para a base de treinamento:
t = 300

#Filtro de língua, escolha uma na tabela ISO 639-1.
#lang = 'pt'
lang = 'en'

Capturando os dados do twitter:

In [5]:
'''
#Cria um objeto para a captura
api = tweepy.API(auth)

#Inicia a captura, para mais detalhes: ver a documentação do tweepy
i = 1
msgs = []
for msg in tweepy.Cursor(api.search, q=produto, lang=lang).items():    
    msgs.append(msg.text.lower())
    i += 1
    if i > n:
        break

#Embaralhando as mensagens para reduzir um possível viés
shuffle(msgs)
'''

'\n#Cria um objeto para a captura\napi = tweepy.API(auth)\n\n#Inicia a captura, para mais detalhes: ver a documentação do tweepy\ni = 1\nmsgs = []\nfor msg in tweepy.Cursor(api.search, q=produto, lang=lang).items():    \n    msgs.append(msg.text.lower())\n    i += 1\n    if i > n:\n        break\n\n#Embaralhando as mensagens para reduzir um possível viés\nshuffle(msgs)\n'

Salvando os dados em uma planilha Excel:

In [6]:
'''
#Verifica se o arquivo não existe para não substituir um conjunto pronto
if not os.path.isfile('./{0}.xlsx'.format(produto)):
    
    #Abre o arquivo para escrita
    writer = pd.ExcelWriter('{0}.xlsx'.format(produto))

    #divide o conjunto de mensagens em duas planilhas
    dft = pd.DataFrame({'Treinamento' : pd.Series(msgs[:t])})
    dft.to_excel(excel_writer = writer, sheet_name = 'Treinamento', index = False)

    dfc = pd.DataFrame({'Teste' : pd.Series(msgs[t:])})
    dfc.to_excel(excel_writer = writer, sheet_name = 'Teste', index = False)

    #fecha o arquivo
    writer.save()
'''

"\n#Verifica se o arquivo não existe para não substituir um conjunto pronto\nif not os.path.isfile('./{0}.xlsx'.format(produto)):\n    \n    #Abre o arquivo para escrita\n    writer = pd.ExcelWriter('{0}.xlsx'.format(produto))\n\n    #divide o conjunto de mensagens em duas planilhas\n    dft = pd.DataFrame({'Treinamento' : pd.Series(msgs[:t])})\n    dft.to_excel(excel_writer = writer, sheet_name = 'Treinamento', index = False)\n\n    dfc = pd.DataFrame({'Teste' : pd.Series(msgs[t:])})\n    dfc.to_excel(excel_writer = writer, sheet_name = 'Teste', index = False)\n\n    #fecha o arquivo\n    writer.save()\n"

___
## Classificando as Mensagens

Agora você deve abrir o arquivo Excel com as mensagens capturadas e classificar na Coluna B se a mensagem é relevante ou não.<br /> 
Não se esqueça de colocar um nome para a coluna na célula **B1**.<br /><br />
Fazer o mesmo na planilha de Controle.

___
## Montando o Classificador Naive-Bayes

Com a base de treinamento montada, comece a desenvolver o classificador. Escreva o seu código abaixo:

Opcionalmente: 
* Limpar as mensagens removendo os caracteres: enter, :, ", ', (, ), etc. Não remover emojis.<br />
* Corrigir separação de espaços entre palavras e/ou emojis.
* Propor outras limpezas/transformações que não afetem a qualidade da informação.



In [7]:
# Carregar a planilha de excel em um formato python
database= pd.read_excel("Drawing Tablet.xlsx")
wordbase= pd.DataFrame(columns=['Palavra', 'Relevante', 'Irrelevante'])

# Ler as mensagens, tokenizando as palavras
#--considerar só palavras com mais de 3 letras (excluir palavras como "a", "of", "the")
#--remover palavras que sejam de outra forma não interessante
# Contar quantas vezes minhas palavras chaves aparecem em tweets relevantes e tweets irrelevantes
size= len(database["Treinamento"])
i=0
while i!=size:
    temp=[]
    for word in database["Treinamento"][i].split():
        if ('@' in word):
            continue #pular palavras que forem nicknames
        word= word.strip()
        if ('http' in word):
            word='#website#' #considerar todos os endereços de rede como um item monolítico
        word = re.sub(r'[^\w\s#]','',word) #tirar pontuações, mantendo hashtags
        if word not in temp:
            temp.append(word) #Assim desconsidero a mesma palavra multiplas vezes no mesmo tweet
            if len(word)>3:
                if (not (wordbase['Palavra'] == word).any()): #se a palavra for nova para meu dataframe...
                    wordbase.loc[-1]= [word, 0, 0]
                    wordbase.index= wordbase.index + 1
                    
                current= wordbase.loc[wordbase['Palavra'] == word].index
                if database["Relevância"][i]=='relevante':
                    wordbase['Relevante'][current]+=1
                if database["Relevância"][i]=='irrelevante':
                    wordbase['Irrelevante'][current]+=1
                    
    i+=1
    
# Excluir palavras com menos de 4 ocorrências (não é suficientemente interessante)
wordbase = wordbase[wordbase['Relevante']+wordbase['Irrelevante'] >= 4]

wordbase.head()

,Palavra,Relevante,Irrelevante
888,drawing,174,97
887,tablet,173,100
886,came,4,0
885,today,6,1
884,#art,4,1


___
## Verificando a performance

Agora você deve testar o seu Classificador com a base de Testes.<br /><br /> 

Você deve extrair as seguintes medidas:
* Porcentagem de positivos falsos (marcados como relevante mas não são relevantes)
* Porcentagem de positivos verdadeiros (marcado como relevante e são relevantes)
* Porcentagem de negativos verdadeiros (marcado como não relevante e não são relevantes)
* Porcentagem de negativos falsos (marcado como não relevante e são relevantes)

Opcionalmente:
* Criar categorias intermediárias de relevância baseado na diferença de probabilidades. Exemplo: muito relevante, relevante, neutro, irrelevante e muito irrelevante.

In [12]:
import time
# Criar um novo arquivo excel com novos tweets, dessa vez categorizados pela máquina

#Verifica se o arquivo não existe para não substituir um conjunto pronto

if not os.path.isfile('./{0}-Final.xlsx'.format(produto)):
    
    #Cria um objeto para a captura
    api = tweepy.API(auth)

    #Inicia a captura, para mais detalhes: ver a documentação do tweepy
    i = 1
    msgs = []
    for msg in tweepy.Cursor(api.search, q=produto, lang=lang).items():    
        msgs.append(msg.text.lower())
        i += 1
        if i > n:
            break
            
    #Abre o arquivo para escrita
    writer = pd.ExcelWriter('./{0}-Final.xlsx'.format(produto))

    #divide o conjunto de mensagens em duas planilhas
    results = pd.DataFrame(columns=["Tweet", "Relevância"])
    
    for tweet in msgs:
        temp=[]
        isrelevant=1.0
        isirrelevant=1.0
        for word in tweet.split():
            if ('@' in word):
                continue #pular palavras que forem nicknames
            word= word.strip()
            if ('http' in word):
                word='#website#' #considerar todos os endereços de rede como um item monolítico
            word = re.sub(r'[^\w\s#]','',word) #tirar pontuações, mantendo hashtags
            if word not in temp:
                temp.append(word) #Assim desconsidero a mesma palavra multiplas vezes no mesmo tweet
                if len(word)>3:
                    ### Este é o bloco em que de fato aplico meu algoritmo ###
                    
                    if ((wordbase['Palavra'] == word).any()): #se a palavra estiver de fato no meu dataframe...      
                        current= wordbase.loc[wordbase['Palavra'] == word].index
                        #probabilidade que a palavra seja relevante, aplicando laplace smoothing
                        prob= int(wordbase['Relevante'][current] + 1)/float(wordbase['Relevante'][current] + wordbase['Irrelevante'][current] + len(wordbase['Palavra']))
                        isrelevant*=prob
                        prob= int(wordbase['Irrelevante'][current] + 1)/float(wordbase['Relevante'][current] + wordbase['Irrelevante'][current] + len(wordbase['Palavra']))
                        isirrelevant*=prob
                        
                    ### Fim do bloco ###
        word="irrelevante"
        if (isrelevant > isirrelevant):
            word='relevante'
        results.loc[-1]= [tweet, word]
        results.index= results.index + 1
    
    results.to_excel(excel_writer = writer, sheet_name = 'Teste', index = False)

    #fecha o arquivo
    writer.save()

In [17]:
#Comparar os resultados obtidos pelo algoritmo com a classificação dada por mim:

# Carregar a planilha de excel em um formato python
database= pd.read_excel("Drawing Tablet-Final.xlsx")
results= pd.DataFrame(columns=['Positivos Falsos', 'Positivos Verdadeiros', 'Negativos Verdadeiros', 'Negativos Falsos'])
results.loc[-1]= [0, 0, 0, 0]
results.index= results.index + 1

# Ler as mensagens, tokenizando as palavras
#--considerar só palavras com mais de 3 letras (excluir palavras como "a", "of", "the")
#--remover palavras que sejam de outra forma não interessante
# Contar quantas vezes minhas palavras chaves aparecem em tweets relevantes e tweets irrelevantes
size= len(database["Tweet"])
i=0

while i!=size:
    
    if(database["Relevância – Máquina"][i]=='relevante' and database["Relevância – Humano"][i]=='relevante'):
        results['Positivos Verdadeiros'][0]+=1
    elif(database["Relevância – Máquina"][i]=='irrelevante' and database["Relevância – Humano"][i]=='relevante'):
        results['Negativos Falsos'][0]+=1
    elif(database["Relevância – Máquina"][i]=='relevante' and database["Relevância – Humano"][i]=='irrelevante'):
        results['Positivos Falsos'][0]+=1
    elif(database["Relevância – Máquina"][i]=='irrelevante' and database["Relevância – Humano"][i]=='irrelevante'):
        results['Negativos Verdadeiros'][0]+=1            
    i+=1

total= results['Positivos Verdadeiros'][0] + results['Negativos Falsos'][0] + results['Positivos Falsos'][0] + results['Negativos Verdadeiros']

print('Porcentagem de Positivos Verdadeiros: '+str(100*results['Positivos Verdadeiros'][0]/float(total))+'%')
print('Porcentagem de Negativos Falsos '+str(100*results['Negativos Falsos'][0]/float(total))+'%')
print('Porcentagem de Positivos Falsos '+str(100*results['Positivos Falsos'][0]/float(total))+'%')
print('Porcentagem de Negativos Verdadeiros '+str(100*results['Negativos Verdadeiros'][0]/float(total))+'%')

results

Porcentagem de Positivos Verdadeiros: 59.8%
Porcentagem de Negativos Falsos 3.6%
Porcentagem de Positivos Falsos 8.0%
Porcentagem de Negativos Verdadeiros 28.6%


,Positivos Falsos,Positivos Verdadeiros,Negativos Verdadeiros,Negativos Falsos
0,40,299,143,18


___
## Concluindo

Escreva aqui a sua conclusão.<br /> 
Faça um comparativo qualitativo sobre as medidas obtidas.<br />
Explique como são tratadas as mensagens com dupla negação e sarcasmo.<br />
Proponha um plano de expansão. Por que eles devem continuar financiando o seu projeto?<br />

Opcionalmente: 
* Discorrer por que não posso alimentar minha base de Treinamento automaticamente usando o próprio classificador, aplicado a novos tweets.
* Propor diferentes cenários de uso para o classificador Naive-Bayes. Cenários sem intersecção com este projeto.
* Sugerir e explicar melhorias reais no classificador com indicações concretas de como implementar (não é preciso codificar, mas indicar como fazer e material de pesquisa sobre o assunto).


As medidas obtidas foram satisfatórias, com um sucesso de medidas de quase 90%. A principal força do algoritmo treinado pareceu ser, com uma faixa grande de sucesso, descartar tweets de spam comerciais,

Um caminho possível a seguir para continuar a desenvolver o aplicativo é tratar retweets diferentemente de tweets originais. Pela natureza que tem esses tendem a cortar palavras, criar biases quanto a endereços de rede e a magnificar exageradamente certos tweets populares.

Sarcasmo e dupla negação não foram tratados, felizmente as amostras analisadas na grande maioria não apresentaram essa espécie de linguagem.

O algoritmo treinado infelizmente não pode se aprimorar por si só, a forma com que funciona requer que suas análises estatísticas das palavras utilizadas existam à priori da aplicão. Por isso é necessário o treinamento por meio da inclusão de uma análise feita por uma pessoa quanto a quais tweets são relevantes ou não

O experimento mostra outras possibilidades para o algoritmo de Naive-Bayes, problemas formulados como uma questão de múltipla decisão baseada em sucesso e fracasso podem ser tratados com o algoritmo de Naive-Bayes, incluindo path-finding para AIs ou o aprendizado de jogos complexos como xadrez 